In [1]:
import sys
sys.path.append('./../src')

# python's shit
from datetime import date, datetime, timedelta
import time
import calendar

# someone elses shit
import finnhub
import pandas

# my shit
import config
from lib.pickle_jar import PickleJar

In [2]:
finnhub_client = finnhub.Client(api_key=config.api_keys['finnhub']['sandbox'])
jar = PickleJar()

In [3]:
# Time junk, just for testing?
now = time.time()
today = date.today()
yesterday = today + timedelta(days = -1)

In [4]:
def recommendation_trends_by_date(ticker, date):
    first_of_month = date.replace(day = 1)
    recommendations = finnhub_client.recommendation_trends(ticker)
    recommendation = [r for r in recommendations if datetime.strptime(r['period'], '%Y-%m-%d').date() == first_of_month]
    if (len(recommendation) > 0):
        return recommendation[0]
    return None

In [5]:
def company_earnings_by_date(ticker, date):
    reporting_period = timedelta(days = 90)
    earnings = finnhub_client.company_earnings(ticker)
    earning = [r for r in earnings if datetime.strptime(r['period'], '%Y-%m-%d').date() + reporting_period > date]
    if (len(earning) > 0):
        return earning[0]
    return None

Build a base data frame from the stock's low, open, close, high and volume for the given time period

In [6]:
def stock_candles_by_date(ticker, date):
    first_of_month_ts = int(datetime.combine(date.replace(day = 1), datetime.min.time()).timestamp())
    next_month = date.replace(day = 28) + timedelta(days = 4)
    last_of_month = next_month - timedelta(days = next_month.day)
    last_of_month_ts = datetime.combine(last_of_month, datetime.max.time()).timestamp()
    earliest = int(min(time.time(), last_of_month_ts))
    candles = finnhub_client.stock_candles('AAPL', '60', first_of_month_ts, earliest)
    return zip(candles['l'], candles['o'], candles['c'], candles['h'], candles['v'])

Construct a set of data frames that each hold:
- low
- open
- close
- high
- volume
- eps_actual (the actual EPS of the last relevant reporting period of the data frame's time period)
- eps_estimate (the estimated EPS, same as above)
- rec_strong_sell
- rec_sell
- rec_hold
- rec_buy
- rec_strong_sell (this and the above 4 fields are proportions of each rec cat of the total [0 - 1])

In [7]:
def construct_data_frames(ticker, date):
    trend = recommendation_trends_by_date(ticker, date)
    trend_total = trend['strongSell'] + trend['sell'] + trend['hold'] + trend['buy'] + trend['strongBuy']
    earnings = company_earnings_by_date(ticker, date)
    static_data = [
        earnings['actual'], earnings['estimate'],
        trend['strongSell'] / trend_total, trend['sell'] / trend_total,
        trend['hold'] / trend_total,
        trend['buy'] / trend_total, trend['strongBuy'] / trend_total
    ]
    base_frames = stock_candles_by_date(ticker, date)
    return [list(x) + static_data for x in base_frames]

Go through the calendar and fetch all the historical data on this ticker that we have access to.
Let's start with 3 months.

In [14]:
def get_historical_data(ticker, days = 90):
    # if this historical pickle file exists, just return it
    historical_pickle_name = "{0}.technical.historical.df.pkl".format(ticker)
    if jar.pickle_exists(historical_pickle_name):
        return jar.read_pickle_dataframe(historical_pickle_name)
    # if it doesn't, build it
    historical_data = []
    today = date.today()
    x_days_ago = today + timedelta(days = -1 * days)
    current_date = x_days_ago
    while current_date < today:
        # look for pickle file for this days data for this day's ticker
        pickle_name = "{0}-{1}.technical.df.pkl".format(ticker, current_date.strftime("%m-%d-%Y"))
        if jar.pickle_exists(pickle_name):
            data = jar.read_pickle_dataframe(pickle_name)
        else:
            data = construct_data_frames(ticker, current_date)
            # pickle this day's data to cut down own API requests
            df = pandas.DataFrame(data)
            jar.write_pickle_dataframe(pickle_name, df)
        historical_data.append(data)
        current_date = current_date + timedelta(days = 1)
    # pickle this historical data
    historical_df = pandas.DataFrame(historical_data)
    jar.write_pickle_dataframe(historical_pickle_name, historical_df)
    return historical_df

In [15]:
data = get_historical_data('AAPL')

/Users/tim/.pyenv/versions/3.8.7/lib/python3.8/site-packages/pandas/core/internals/construction.py:309: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  values = np.array([convert(v) for v in values])


In [16]:
data.head()

,0
0,0 1 2 ...
1,0 1 2 ...
2,0 1 2 ...
3,0 1 2 ...
4,0 1 2 ...


In [13]:
data_frames_02_24_22 = jar.read_pickle_dataframe("{0}-{1}.technical.df.pkl".format('AAPL', yesterday.strftime("%m-%d-%Y")))

In [14]:
data_frames_02_24_22.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,122.631000,123.000000,122.815500,123.492000,69,1.68,1.555857,0.024194,0.024194,0.174731,0.5,0.276882
1,122.385738,122.754000,122.569869,123.245016,83,1.68,1.555857,0.024194,0.024194,0.174731,0.5,0.276882
2,122.140967,122.508492,122.324729,122.998526,91,1.68,1.555857,0.024194,0.024194,0.174731,0.5,0.276882
3,121.896685,122.263475,122.544681,122.752529,36,1.68,1.555857,0.024194,0.024194,0.174731,0.5,0.276882
4,122.445220,122.813661,123.096132,123.304915,97,1.68,1.555857,0.024194,0.024194,0.174731,0.5,0.276882
